# Calculos preliminares - Censo - nivel regional
El objetivo de este notebook es ocupar el dataset del cesnso preprocesado para obtener metricas estadisticas que se ocuparan en el estudio.

## Importar librerias
Inicialmente importamos las librerias necesarias para el funcionamiento del notebook.

In [1]:
import pandas as pd

## Asignacion nombres de archivo
Para leer los archivos y posteriormente exportarlos a otro csv debemos asignarle el nombre de lectura correspondiente

In [2]:
# Nombrar segmento de archivo
regiones = ["1_Tarapacá",
            "2_Antofagasta",
            "3_Atacama",
            "4_Coquimbo",
            "5_Valparaíso",
            "6_Libertador General Bernardo O'Higgins",
            "7_Maule",
            "8_Biobío",
            "9_La Araucanía",
            "10_Los Lagos",
            "11_Aysén del General Carlos Ibáñez del Campo",
            "12_Magallanes y de la Antártica Chilena",
            "13_Metropolitana de Santiago",
            "14_Los Ríos",
            "15_Arica y Parinacota",
            "16_Ñuble"
]

## Lectura csv inmigracion y poblacion

In [3]:
## csv poblacion
dfs_poblacion = []
for region in regiones:
    ruta = f"../../data/processed_data/censo/censo_poblacion/poblacion_comunas_{region}.csv"
    df = pd.read_csv(ruta)
    dfs_poblacion.append(df)

In [4]:
## csv inmigracion
dfs_inmigracion = []
for region in regiones:
    ruta = f"../../data/processed_data/censo/censo_inmigracion/inmigracion_comunas_{region}.csv"
    df = pd.read_csv(ruta)
    dfs_inmigracion.append(df)

## Se une cada csv de inmigracion con el respectivo de poblacion

In [13]:
## mergear todo
merged_dfs = []
for i in range(len(dfs_poblacion)):
    df_poblacion = dfs_poblacion[i][["Código región","Región","Provincia","Código comuna","Comuna","Población censada"]]
    df_inmigracion = dfs_inmigracion[i][["Código región","Región","Provincia","Codigo comuna","Comuna","País o continente de nacimiento","Inmigrantes internacionales"]]
    df_poblacion = df_poblacion.rename(columns={"Población censada": "poblacion_total"})
    df_inmigracion = df_inmigracion.rename(columns={"Inmigrantes internacionales": "poblacion_migrante"})
    df_inmigracion = df_inmigracion.rename(columns={"Codigo comuna": "Código comuna"})
    
    ##limpieza
    df_inmigracion["poblacion_migrante"] = df_inmigracion["poblacion_migrante"].replace("*", 0).astype(int)

    
    df_merged = df_poblacion.merge(df_inmigracion, on=["Código región", "Región","Provincia","Código comuna","Comuna"])
    df_merged['Código comuna'] = df_merged['Código comuna'].astype(int)
    merged_dfs.append(df_merged)
    

## Selección y cálculo de metricas
Se selecciona el total de poblacion migrante, y se calcula el porcentaje de poblacion migrante respecto a la poblacion total

In [14]:
## resumen inmigracion por comuna total!
resumen_total = []
for df in merged_dfs:
    df_filtrado = df[df["País o continente de nacimiento"] == "Total nacidos fuera del país"].copy()
    df_filtrado["porcentaje"] = round(df_filtrado["poblacion_migrante"] / df_filtrado["poblacion_total"] * 100,2)
    resumen_total.append(df_filtrado)


## Exportacion a csv
Finalmente generamos los csv para ser utilizado en el estudio, estos se guardan en el mismo formato que se obtuvieron con los nombres especificados al principio del notebook

In [16]:
for df in resumen_total:
    nombre = df["Región"].iloc[0]
    codigo = df["Código región"].iloc[0]
    df.to_csv(f'../../data/processed_data/preliminar_estadisticas/region/region_total/estadisticas_total_comunas_{codigo}_{nombre}.csv', index=False)